In [1]:
import sys
import pandas as pd
sys.path.insert(0, ".././0.data-download/scripts/")
from data_loader import load_data
from scipy.stats import ttest_ind

In [2]:
latent_df = pd.read_csv("../2.train-VAE/results/latent_df.csv")
metadata_df = pd.read_csv(".././0.data-download/data/metadata_df.csv")
data_dir = "../0.data-download/data/"
model_df, dependency_df = load_data(data_dir, adult_or_pediatric="all")


In [3]:
# Creating categorized lists of sample IDs used in BVAE training
# note that 10 of the 912 used samples have Unknown Sex

ped_ids = []
adult_ids = []
male_ids = []
female_ids = []
ped_male_ids = []
adult_male_ids = []
ped_female_ids = []
adult_female_ids = []

for index, row in metadata_df.iterrows():
    if row['AgeCategory'] == 'Pediatric':
        ped_ids.append(row['ModelID'])
    
    if row['AgeCategory'] == 'Adult':
        adult_ids.append(row['ModelID'])
    
    if row['Sex'] == 'Male':
        male_ids.append(row['ModelID'])

    if row['Sex'] == 'Female':
        female_ids.append(row['ModelID'])
    
    if row['AgeCategory'] == 'Pediatric' and row['Sex'] == 'Male':
        ped_male_ids.append(row['ModelID'])

    if row['AgeCategory'] == 'Adult' and row['Sex'] == 'Male':
        adult_male_ids.append(row['ModelID'])
    
    if row['AgeCategory'] == 'Pediatric' and row['Sex'] == 'Female':
        ped_female_ids.append(row['ModelID'])

    if row['AgeCategory'] == 'Adult' and row['Sex'] == 'Female':
        adult_female_ids.append(row['ModelID'])


In [4]:
# Generating latent dataframes for each category and dropping the id column to prep for t tests

adult_latent_df = latent_df.copy()
for index, row in adult_latent_df.iterrows():
    if row['ModelID'] not in adult_ids:
        adult_latent_df.drop(index, inplace=True)
adult_latent_df_float = adult_latent_df.drop(columns=["ModelID"])
adult_latent_df_float.reset_index(drop=True, inplace=True)

ped_latent_df = latent_df.copy()
for index, row in ped_latent_df.iterrows():
    if row['ModelID'] not in ped_ids:
        ped_latent_df.drop(index, inplace=True)
ped_latent_df_float = ped_latent_df.drop(columns=["ModelID"])
ped_latent_df_float.reset_index(drop=True, inplace=True)

male_latent_df = latent_df.copy()
for index, row in male_latent_df.iterrows():
    if row['ModelID'] not in male_ids:
        male_latent_df.drop(index, inplace=True)
male_latent_df_float = male_latent_df.drop(columns=["ModelID"])
male_latent_df_float.reset_index(drop=True, inplace=True)

female_latent_df = latent_df.copy()
for index, row in female_latent_df.iterrows():
    if row['ModelID'] not in female_ids:
        female_latent_df.drop(index, inplace=True)
female_latent_df_float = female_latent_df.drop(columns=["ModelID"])
female_latent_df_float.reset_index(drop=True, inplace=True)

ped_male_latent_df = latent_df.copy()
for index, row in ped_male_latent_df.iterrows():
    if row['ModelID'] not in ped_male_ids:
        ped_male_latent_df.drop(index, inplace=True)
ped_male_latent_df_float = ped_male_latent_df.drop(columns=["ModelID"])
ped_male_latent_df_float.reset_index(drop=True, inplace=True)

adult_male_latent_df = latent_df.copy()
for index, row in adult_male_latent_df.iterrows():
    if row['ModelID'] not in adult_male_ids:
        adult_male_latent_df.drop(index, inplace=True)
adult_male_latent_df_float = adult_male_latent_df.drop(columns=["ModelID"])
adult_male_latent_df_float.reset_index(drop=True, inplace=True)

ped_female_latent_df = latent_df.copy()
for index, row in ped_female_latent_df.iterrows():
    if row['ModelID'] not in ped_female_ids:
        ped_female_latent_df.drop(index, inplace=True)
ped_female_latent_df_float = ped_female_latent_df.drop(columns=["ModelID"])
ped_female_latent_df_float.reset_index(drop=True, inplace=True)

adult_female_latent_df = latent_df.copy()
for index, row in adult_female_latent_df.iterrows():
    if row['ModelID'] not in adult_female_ids:
        adult_female_latent_df.drop(index, inplace=True)
adult_female_latent_df_float = adult_female_latent_df.drop(columns=["ModelID"])
adult_female_latent_df_float.reset_index(drop=True, inplace=True)

In [5]:
# t tests comparing adult vs ped for each latent dimension

t_test_adult_vs_ped = ttest_ind(adult_latent_df_float, ped_latent_df_float)
t_test_adult_vs_ped = pd.DataFrame(t_test_adult_vs_ped).T
t_test_adult_vs_ped.columns = ["t_stat", "p_value"]
print(t_test_adult_vs_ped.shape)

# Sort to show latent dimensions with most significant p values
t_test_adult_vs_ped.sort_values(by='p_value', ascending = True)

(70, 2)


,t_stat,p_value
49,-3.921116,0.000095
42,-2.780591,0.005538
44,2.566997,0.010417
40,2.484561,0.013150
17,-2.336992,0.019656
...,...,...
41,0.071369,0.943120
63,0.028125,0.977568
43,0.020142,0.983934
38,-0.018726,0.985064


In [6]:
# t tests comparing male vs female for each latent dimension

t_test_male_vs_female = ttest_ind(male_latent_df_float, female_latent_df_float)
t_test_male_vs_female = pd.DataFrame(t_test_male_vs_female).T
t_test_male_vs_female.columns = ["t_stat", "p_value"]
print(t_test_male_vs_female.shape)

# Sort to show latent dimensions with most significant p values
t_test_male_vs_female.sort_values(by='p_value', ascending = True)

(70, 2)


,t_stat,p_value
14,-3.318277,0.000942
35,2.397202,0.016724
25,-2.269413,0.023479
27,-1.845678,0.065267
41,1.806101,0.071237
...,...,...
39,0.088209,0.929730
64,-0.045838,0.963449
30,0.039444,0.968545
36,-0.032841,0.973809


In [7]:
# t tests comparing adult male vs ped male for each latent dimension

t_test_adult_male_vs_ped_male = ttest_ind(ped_male_latent_df_float, adult_male_latent_df_float)
t_test_adult_male_vs_ped_male = pd.DataFrame(t_test_adult_male_vs_ped_male).T
t_test_adult_male_vs_ped_male.columns = ["t_stat", "p_value"]
print(t_test_adult_male_vs_ped_male.shape)

# Sort to show latent dimensions with most significant p values
t_test_adult_male_vs_ped_male.sort_values(by='p_value', ascending = True)

(70, 2)


,t_stat,p_value
49,3.017996,0.002671
2,-2.788986,0.005483
44,-2.465013,0.014027
59,2.222444,0.026688
17,2.207817,0.027698
...,...,...
21,0.062437,0.950239
61,0.061975,0.950607
66,0.046168,0.963194
58,0.028072,0.977615


In [8]:
# t tests comparing adult female vs ped female for each latent dimension

t_test_adult_female_vs_ped_female = ttest_ind(ped_female_latent_df_float, adult_female_latent_df_float)
t_test_adult_female_vs_ped_female = pd.DataFrame(t_test_adult_female_vs_ped_female).T
t_test_adult_female_vs_ped_female.columns = ["t_stat", "p_value"]
print(t_test_adult_female_vs_ped_female.shape)

# Sort to show latent dimensions with most significant p values
t_test_adult_female_vs_ped_female.sort_values(by='p_value', ascending = True)

(70, 2)


,t_stat,p_value
49,3.320037,0.000986
29,-2.439531,0.015158
4,2.328252,0.020417
42,2.300494,0.021956
46,-2.127966,0.033976
...,...,...
22,-0.138869,0.889627
5,0.102093,0.918737
31,0.070552,0.943791
43,0.062207,0.950430


In [9]:
# t tests comparing ped male vs ped female for each latent dimension

t_test_ped_male_vs_ped_female = ttest_ind(ped_female_latent_df_float, ped_male_latent_df_float)
t_test_ped_male_vs_ped_female = pd.DataFrame(t_test_ped_male_vs_ped_female).T
t_test_ped_male_vs_ped_female.columns = ["t_stat", "p_value"]
print(t_test_ped_male_vs_ped_female.shape)

# Sort to show latent dimensions with most significant p values
t_test_ped_male_vs_ped_female.sort_values(by='p_value', ascending = True)

(70, 2)


,t_stat,p_value
45,2.945469,0.003832
7,-2.737408,0.007074
16,-2.506251,0.013455
9,-2.384078,0.018590
13,-2.217199,0.028374
...,...,...
68,-0.075212,0.940163
37,-0.062952,0.949903
12,0.049301,0.960756
32,-0.034931,0.972189


In [10]:
# t tests comparing adult male vs adult female for each latent dimension

t_test_adult_male_vs_adult_female = ttest_ind(adult_female_latent_df_float, adult_male_latent_df_float)
t_test_adult_male_vs_adult_female = pd.DataFrame(t_test_adult_male_vs_adult_female).T
t_test_adult_male_vs_adult_female.columns = ["t_stat", "p_value"]
print(t_test_adult_male_vs_adult_female.shape)

# Sort to show latent dimensions with most significant p values
t_test_adult_male_vs_adult_female.sort_values(by='p_value', ascending = True)

(70, 2)


,t_stat,p_value
14,3.828278,0.000140
41,-2.404327,0.016437
35,-2.268182,0.023594
27,2.167195,0.030526
25,2.054542,0.040260
...,...,...
49,-0.115032,0.908450
11,0.080381,0.935955
8,-0.071901,0.942699
62,-0.035133,0.971983
